### Gruppo 5.a: Castagnacci Giulia, Giordano Elisabetta

### Analisi congruence

In [1]:

using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using IntervalTrees
using SparseArrays
using NearestNeighbors
using BenchmarkTools
using OrderedCollections
using Base.Threads


### Dati in input

In [2]:
V = hcat([[0.,0],[1,0],[1,1],[0,1],[2,1]]...);                                          #vertici del modello 2D
V3 = hcat([[0.,0,0],[1,0,3],[1,1,2],[0,1,1],[2,1,0]]...);                               #vertici del modello 3D
EV = [[1,2],[2,3],[3,4],[4,1],[1,5]];                                                   #spigoli del modello
bb = [[0.0 1.0; 0.0 0.0], [1.0 1.0; 0.0 1.0], [0.0 1.0; 1.0 1.0], [0.0 0.0; 0.0 1.0], [0.0 2.0; 0.0 1.0]];  #bounding box
dict = OrderedDict([0.0, 1.0] => [1, 3],[1.0, 1.0] => [2],[0.0, 0.0] => [4],[0.0, 2.0] => [5])  #dizionario intervallo/indice
cov = [[4, 1, 3, 5, 2], [1, 3, 5, 2], [4, 1, 3, 5, 2], [4, 1, 3, 5], [4, 1, 3, 5, 2]]    #intersezioni tra bounding box

5-element Vector{Vector{Int64}}:
 [4, 1, 3, 5, 2]
 [1, 3, 5, 2]
 [4, 1, 3, 5, 2]
 [4, 1, 3, 5]
 [4, 1, 3, 5, 2]

### Versione iniziale congruence

Funzione che prende in input un model di tipo Lar.LAR, inizializza un BallTree, che divide ricorsivamente i punti in gruppi delimitati da ipersfere, un raggio di ricerca e un array vuoto di W elementi (W = matrice 2xW). Per ogni vertice cerca i vertici più vicini nel raggio R e li sostituisce. Crea un dizionario chiave, valore (id nuovi vertici) che verrà poi utilizzato per etichettare i vertici degli spigoli in EW.

In [3]:
function congruence(model)
	W,EW = model
	# congruent vertices
	balltree = NearestNeighbors.BallTree(W)
	r = 0.0000000001
	near = Array{Any}(undef, size(W,2))
	for k=1:size(W,2)
		near[k] = union([NearestNeighbors.inrange(balltree, W[:,k], r, true)]...)
	end
	near = map(sort,near)  # check !!!
	for k=1:size(W,2)
		W[:,k] = W[:,near[k][1]]
	end
	pointidx = [ near[k][1] for k=1:size(W,2) ]  # check !!
	invidx = OrderedDict(zip(1:length(pointidx), pointidx))
	V = [W[:,k] for k=1:length(pointidx)]
	# congruent edges
	EV = []
	for e in (EW)
		newedge = [invidx[e[1]],invidx[e[2]]]
		if newedge[1] !== newedge[2]
			push!(EV,newedge)
		end
	end
	EV = [EV[h] for h=1:length(EV) if length(EV[h])==2]
	EV = convert(Lar.Cells, EV)
	
	return hcat(V...),EV
end

congruence (generic function with 1 method)

### Analisi del comportamento e dei tempi della versione iniziale

In [4]:
@btime congruence((V,EV))

  24.500 μs (176 allocations: 9.83 KiB)


([0.0 1.0 … 0.0 2.0; 0.0 0.0 … 1.0 1.0], [[1, 2], [2, 3], [3, 4], [4, 1], [1, 5]])

In [5]:
@code_warntype congruence((V,EV))

MethodInstance for congruence(

::Tuple{Matrix{Float64}, Vector{Vector{Int64}}})
  from congruence(model) in Main at c:\Users\giord\eclipse-SIW\LARSplitting2D\notebooks\refactoringAnalisi_congruence.ipynb:1


Arguments
  #self#

::Core.Const(congruence)
  model::Tuple{Matrix{Float64}, Vector{Vector{Int64}}}
Locals
  #12::var"#12#16"
  #11::var"#11#15"
  @_5::Union{Nothing, Tuple{Vector{Int64}, Int64}}
  #10::var"#10#14"{Matrix{Float64}}
  #9::var"#9#13"
  @_8::Union{Nothing, Tuple{Int64, Int64}}
  @_9::Union{Nothing, Tuple{Int64, Int64}}
  @_10::Int64
  EV@_11::Core.Box
  V::Vector{Vector{Float64}}
  invidx::OrderedDict
  pointidx::Vector
  near@_15::Core.Box
  r::Float64
  balltree::BallTree{_A, _B, _C, Euclidean} where {_A<:(AbstractVector), _B, _C}
  EW::Vector{Vector{Int64}}
  W::Matrix{Float64}
  k@_20::Int64
  k@_21::Int64
  e::Vector{Int64}
  newedge::Vector
  near@_24::Union{}
  near@_25::Union{}
  near@_26::Union{}
  EV@_27::Union{}
  EV@_28::Union{}
  EV@_29::Union{}
  EV@_30::Union{}
Body::Tuple{Union{Matrix, Vector{Any}}, Any}


1 ──        

...

@_5::Tuple{Vector{Int64}, Int64}
│           (e = Core.getfield(%111, 1))
│    %113 = Core.getfield(%111, 2)::Int64
│    %114 = invidx::OrderedDict
│    %115 = Base.getindex(e, 1)::Int64
│    %116 = Base.getindex(%114, %115)::Any
│    %117 = invidx::OrderedDict
│    %118 = Base.getindex(e, 2)::Int64
│    %119 = Base.getindex(%117, %118)::Any
│           (newedge = Base.vect(%116, %119))
│    %121 = Base.getindex(newedge, 1)::Any
│    %122 = Base.getindex(newedge, 2)::Any
│    %123 = (%121 !== %122)::Bool
└───        goto #22 if not %123
18 ─ %125 = Core.isdefined(EV@_11, :contents)::Bool
└─── 

       goto #20 if not %125
19 ─        goto #21
20 ─        Core.NewvarNode(:(EV@_27))
└───        EV@_27
21 ┄ %130 = Core.getfield(EV@_11, :contents)::Any
└───        Main.push!(%130, newedge)
22 ┄        (@_5 = Base.iterate(%106, %113))
│    %133 = (@_5 === nothing)::Bool
│    %134 = Base.not_int(%133)::Bool
└───        goto #24 if not %134
23 ─        goto #17
24 ┄        (#11 = %new(Main.:(var"#11#15"), EV@_11))
│    %138 = #11::var"#11#15"
│           (#12 = %new(Main.:(var"#12#16"), EV@_11))
│    %140 = #12::var"#12#16"
│    %141 = Core.isdefined(EV@_11, :contents)::Bool
└───        goto #26 if not %141
25 ─        goto #27
26 ─        Core.NewvarNode(:(EV@_28))
└───        EV@_28
27 ┄ %146 = Core.getfield(EV@_11, :contents)::Any
│    %147 = Main.length(%146)::Any
│    %148 = (1:%147)::Any
│    %149 = Base.Filter(%140, %148)::Base.Iterators.Filter{var"#12#16"}
│    %150 = Base.Generator(%138

, %149)::Base.Generator{_A, var"#11#15"} where _A
│    %151 = Base.collect(%150)::Any
│           Core.setfield!(EV@_11, :contents, %151)
│    %153 = Base.getproperty(Main.Lar, :Cells)::Any
│    %154 = Core.isdefined(EV@_11, :contents)::Bool
└───        goto #29 if not %154
28 ─        goto #30
29 ─        Core.NewvarNode(:(EV@_29))
└───        EV@_29
30 ┄ %159 = Core.getfield(EV@_11, :contents)::Any
│    %160 = Main.convert(%153, %159)::Any
│           Core.setfield!(EV@_11, :contents, %160)
│    

%162 = Core._apply_iterate(Base.iterate, Main.hcat, V)::Union{Matrix, Vector{Any}}
│    %163 = Core.isdefined(EV@_11, :contents)::Bool
└───        goto #32 if not %163
31 ─        goto #33
32 ─        Core.NewvarNode(:(EV@_30))
└───        EV@_30
33 ┄ %168 = Core.getfield(EV@_11, :contents)::Any
│    %169 = Core.tuple(%162, %168)

::Tuple{Union{Matrix, Vector{Any}}, Any}
└───        return %169



In [6]:
@benchmark congruence((V,EV))

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  24.800 μs …  21.992 ms  ┊ GC (min … max): 0.00% … 98.84%
 Time  (median):     35.150 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   46.728 μs ± 238.937 μs  ┊ GC (mean ± σ):  6.61% ±  1.40%

  ▁▂▂         █                                                 
  █████▆▄▃▃▂▂▄█▅▂▁▁▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▂▂▁▁▁▁▁▂▅▅▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁ ▂
  24.8 μs         Histogram: frequency by time          103 μs <

 Memory estimate: 9.83 KiB, allocs estimate: 176.

### Versione parallelizzata di congruence
abbiamo paralellizzato la funzione attraverso le macro  @inbounds e @threads. Si nota un certo miglioramento nelle performance se al posto di usare la list comprehension per ottenere i dati di EV validi si fa un filter. Abbiamo quindi convertito alcune list comprehension in cicli del tipo for i=1:n,  in modo da poter utilizzare la macro @inbounds per disabilitare il boundchecking del compilatore.

In [7]:
function congruence2(model)
    W,EW = model
    n = size(W,2)
    balltree = NearestNeighbors.BallTree(W)
    r = 0.0000000001
    near = Array{Any}(undef, n)
    @inbounds @threads for k=1:n
        near[k] = NearestNeighbors.inrange(balltree, W[:,k], r, true)
    end
    near = map(sort,near) 
    @inbounds @threads for k=1:n
        W[:,k] = W[:,near[k][1]]
    end
    pointidx = Array{Int64}(undef, n)
    @inbounds @threads for k=1:n
         pointidx[k] = near[k][1] 
    end
    l = length(pointidx)
    invidx = OrderedDict(zip(1:l, pointidx))
    V = Array{Array{Float64,1}}(undef, l)
    @inbounds @threads for k=1:l
        V[k] = W[:,k] 
    end
    
    EV = []
    m = length(EW)
    @inbounds for i = 1:m
        newedge = [invidx[EW[i][1]],invidx[EW[i][2]]]
        if newedge[1] !== newedge[2]
            push!(EV,newedge)
        end
    end
    filter!(x ->  length(x)==2, EV)
    EV = convert(Lar.Cells, EV)
    return hcat(V...),EV
end

congruence2 (generic function with 1 method)

### Analisi del comportamento e dei tempi della versione parallelizzata

In [8]:
@btime congruence2((V,EV))

  30.800 μs (152 allocations: 9.27 KiB)


([0.0 1.0 … 0.0 2.0; 0.0 0.0 … 1.0 1.0], [[1, 2], [2, 3], [3, 4], [4, 1], [1, 5]])

In [9]:
@benchmark congruence2((V,EV))

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  30.600 μs …  13.427 ms  ┊ GC (min … max): 0.00% … 98.90%
 Time  (median):     33.500 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   41.005 μs ± 163.064 μs  ┊ GC (mean ± σ):  5.48% ±  1.40%

  ▄█▇▆▅▄▄▃▂▃▂▃▂▂▂▂▂▁▁▁▂▁  ▁▃▃▃▂▂▂▂▂▁▁▁▁▁                       ▂
  █████████████████████████████████████████▇▇▇▇▇▅▆▆▅▆▅▆▄▄▄▅▅▄▅ █
  30.6 μs       Histogram: log(frequency) by time      77.9 μs <

 Memory estimate: 9.27 KiB, allocs estimate: 152.

## Test

In [10]:
using Test

@testset "congruence Tests" begin
    V1 = [0.0 1.0 ; 
         0.0 0.0]
    EV1 = [[1,2]]
    a, b = Lar.congruence((V1,EV1))
    @test a == V1
    @test b == EV1

    V1 = [0.0 1.0 0.99999999999999999 ; 0.0 0.0 0.0]
    EV1 = [[1,2], [1,3]]
    a, b = Lar.congruence((V1,EV1))
    @test b == [[1, 2], [1, 2]] 
    
end

Test Summary:    | Pass  Total
congruence Tests |    3      3


Test.DefaultTestSet("congruence Tests", Any[], 3, false, false)

![](https://github.com/GiuliaCastagnacci/LARSplitting2D/blob/main/docs/plot/screenTest/test_congruence.png?raw=true)

![](https://github.com/GiuliaCastagnacci/LARSplitting2D/blob/main/docs/plot/images/test/test_congruence.png?raw=true)
